In [1]:
from publink import publink
import requests 
import pandas as pd 
import os 
from pprintjson import pprintjson as ppjson 

In [2]:
# Spreadsheet with pub in xdd added to it 
fname = os.path.join("pub_in_xdd.csv")
pub_in_xdd = pd.read_csv(fname)
dr_doi_pub = pub_in_xdd
dr_doi_pub

,Unnamed: 0,pub_in_xdd,rel_pub_url,dr_doi
0,0,False,https://doi.org/10.1021/acs.est.8b07227,doi:10.5066/P90QU56J
1,1,False,https://doi.org/10.1007/s13157-017-0895-3,doi:10.5066/F73R0R24
2,2,True,https://doi.org/10.1002/etc.3391,doi:10.5066/F7VQ30RM
3,3,True,https://doi.org/10.3133/sim3378,doi:10.5066/F71G0JF6
4,4,True,https://doi.org/10.3133/sir20155164,doi:10.5066/F7571931
...,...,...,...,...
1349,1349,False,https://doi.org/10.1186/s40462-019-0178-0,doi:10.5066/P9D5IP0G
1350,1350,True,https://doi.org/10.3133/sir20175135,doi:10.5066/F7JH3KBD
1351,1351,False,https://doi.org/10.1371/journal.pone.0197584,doi:10.5066/P9V9AORH
1352,1352,True,https://doi.org/10.3133/sim3423,doi:10.5066/P9BS882S


In [3]:
# Dropped the unnamed column 
dr_doi_pub = dr_doi_pub.drop(columns=['Unnamed: 0'], axis=1)

In [4]:
# Strip url prefix from rel_pub_url column
dr_doi_pub["rel_pub_doi"] = dr_doi_pub["rel_pub_url"].str.replace("https://doi.org/", "")

In [5]:
# Strip "doi:" prefix to the data release DOI
dr_doi_pub["drdoi"] = dr_doi_pub["dr_doi"].str.replace("doi:", "")

In [6]:
# Remove the rel_pub_url and dr_doi columns 
dr_doi_pub = dr_doi_pub.drop(columns=["rel_pub_url", "dr_doi"], axis=1)

In [7]:
dr_doi_pub

,pub_in_xdd,rel_pub_doi,drdoi
0,False,10.1021/acs.est.8b07227,10.5066/P90QU56J
1,False,10.1007/s13157-017-0895-3,10.5066/F73R0R24
2,True,10.1002/etc.3391,10.5066/F7VQ30RM
3,True,10.3133/sim3378,10.5066/F71G0JF6
4,True,10.3133/sir20155164,10.5066/F7571931
...,...,...,...
1349,False,10.1186/s40462-019-0178-0,10.5066/P9D5IP0G
1350,True,10.3133/sir20175135,10.5066/F7JH3KBD
1351,False,10.1371/journal.pone.0197584,10.5066/P9V9AORH
1352,True,10.3133/sim3423,10.5066/P9BS882S


In [8]:
def xdd_pub_mention_dr_doi(dr_doi, pub_doi): # Change row to dr_doi, add pub_doi as param
    url = f"https://geodeepdive.org/api/articles?doi=&max=1"
    r = requests.get(url)
    try:
        search = publink.search_xdd(
        dr_doi, account_for_spaces=True
        )
        mention = publink.xdd_mentions(
        search.response_data, search.search_terms,
        search_type='exact_match', is_doi=True # Change 'row' to 'exact_match'
        )
        # Create a list of pubs mentions
        pubs = [pub['pub_doi'] for pub in mention.mentions]
    except:
        return None 
    return pub_doi in pubs
        

In [9]:
xdd_pub_mention_dr_doi("10.5066/F7765D7V",'10.3133/SIR20195106')

True

In [ ]:
dr_doi_pub['dr_doi_mentioned'] = dr_doi_pub.apply(lambda row: None if row['pub_in_xdd'] == False else xdd_pub_mention_dr_doi(row['drdoi'],row['rel_pub_doi']), axis=1)

In [ ]:
display(dr_doi_pub)

In [ ]:
dr_doi_pub.dr_doi_mentioned.value_counts()